<a href="https://colab.research.google.com/github/aditiprashant07/BUSI-and-LoRA-Project-/blob/main/LoRAandBUSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""Google Collabs runtime is temporary and to ensure it persists after disconnection. Without this you lose everything."""
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
"""This is the best practice to organize data"""
import os

PROJECT_ROOT = "/content/drive/MyDrive/BUSI_LoRA_Project"

folders = ["data", "checkpoints", "outputs"]

for folder in folders:
    os.makedirs(os.path.join(PROJECT_ROOT, folder), exist_ok=True)

print("Project folders created successfully!")


In [ ]:
"""To check if torch and the specific GPU type is available"""
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))


In [ ]:
"""Transformers -> Hugging Face pre-trained library
peft -> Parameter-Efficient Fine Tuning which enables LoRA
scikit-learn -> Metrics, Confusion Matrix, Classification report
Matplotlib -> For plotting training curves
GRAD-CAM -> Explainability - showing what the model sees"""
!pip install transformers
!pip install -q peft
!pip install -q scikit-learn
!pip install -q matplotlib

In [ ]:
#Installing Kaggle from where our dataset will be pulled
!pip install -q kaggle


In [ ]:
#This is important to create a kaggle.json file and for connecting to your Kaggle account.
import json
import os

kaggle_username = "username"
kaggle_key = "token"

kaggle_json = {
    "username": kaggle_username,
    "key": kaggle_key
}

os.makedirs("/root/.kaggle", exist_ok=True)

with open("/root/.kaggle/kaggle.json", "w") as f:
    json.dump(kaggle_json, f)

os.chmod("/root/.kaggle/kaggle.json", 600)

print("kaggle.json created successfully!")


In [ ]:
!kaggle datasets list -s breast-ultrasound


In [ ]:
!kaggle datasets download -d aryashah2k/breast-ultrasound-images-dataset


In [ ]:
!unzip breast-ultrasound-images-dataset.zip -d data/


In [ ]:
"""In each transformation of image we have a resize since ViT an DeiT require a 224x224 ImageNet Images.
After resizing the Image is converted into a tensor value between 0 and 1.
The Normalization is used at the end since ViT was trained on these standard values. Without this the model sees data which is out of it's distribution"""
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# ImageNet normalization values (required for pretrained ViT)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
"""ImageFolder automatically assigns ;abels based on folder names. This is to ensure data remains consistent"""
dataset_path = "data/Dataset_BUSI_with_GT"

full_dataset = datasets.ImageFolder(
    root=dataset_path,
    transform=transform
)

print("Total images:", len(full_dataset))
print("Classes:", full_dataset.classes)


In [ ]:
"""This is standard pattern for small datasets and a random split ensures random distributions"""
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

print("Train size:", len(train_dataset))
print("Validation size:", len(val_dataset))


In [ ]:
#Create Data Loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)


In [ ]:
#Testing One Batch to check if pre processing works
images, labels = next(iter(train_loader))
print("Image batch shape:", images.shape)
print("Label batch shape:", labels.shape)


In [ ]:
#Loading our vision transformer-facebook/deit-small-patch16-224
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "facebook/deit-small-patch16-224"

processor = AutoImageProcessor.from_pretrained(model_name)

model = AutoModelForImageClassification.from_pretrained(
    model_name,
    num_labels=3,
    ignore_mismatched_sizes=True # Add this line to handle the mismatch
)

model.to(device)

print("Model loaded successfully.")

In [ ]:
# This is a sanity check
outputs = model(images.to(device))
print(outputs.logits.shape)


In [ ]:
# Now we begin training the LoRA layers and the Classification Heads- We also need to freeze the base model in this step
for param in model.base_model.parameters():
    param.requires_grad = False

print("Base model frozen.")


In [ ]:
# We apply LoRA to the attention layers
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none"
    # Removed: task_type="IMAGE_CLASSIFICATION" as it's not a valid type
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

In [ ]:
# Define Optimizer and Loss
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=5e-5
)



In [ ]:
scaler = torch.cuda.amp.GradScaler()


In [ ]:
# Create a clean and simple training loop
epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs.logits, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader)}")



In [ ]:
# Run this for validation accuracy
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.logits, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("Validation Accuracy:", 100 * correct / total, "%")


In [ ]:
# Creating a Confusion Matrix
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.logits, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=full_dataset.classes))


In [ ]:
# Install Grad-CAM for medical imaging explainability
!pip install grad-cam -q


In [ ]:
#Import the required libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget


In [ ]:
# Put model in EVAL mode
model.eval()


In [ ]:
# Define a Reshape Function
def reshape_transform(tensor):
    # Remove class token
    tensor = tensor[:, 1:, :]

    h = w = 14  # because image is 224 and patch size is 16 (224/16=14)

    tensor = tensor.reshape(tensor.size(0), h, w, tensor.size(2))
    tensor = tensor.permute(0, 3, 1, 2)  # (B, C, H, W)

    return tensor


In [ ]:
# Wrap the model
class ModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return self.model(x).logits

wrapped_model = ModelWrapper(model)
wrapped_model.eval()


In [ ]:
# Select correct target layer
target_layer = model.base_model.model.vit.encoder.layer[-1].layernorm_after


In [ ]:
# Initialize GRAD-CAM
cam = GradCAM(
    model=wrapped_model,
    target_layers=[target_layer],
    reshape_transform=reshape_transform
)


In [ ]:
# Get one validation image
images, labels = next(iter(val_loader))

image = images[0].unsqueeze(0).to(device)
label = labels[0].item()


In [ ]:
# Generate GRAD-CAM heatmap
targets = [ClassifierOutputTarget(label)]

grayscale_cam = cam(input_tensor=image, targets=targets)
grayscale_cam = grayscale_cam[0]


In [ ]:
# Undo normalization
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

img = images[0].permute(1, 2, 0).cpu().numpy()
img = std * img + mean
img = np.clip(img, 0, 1)


In [ ]:
# Overlap Heatmap
visualization = show_cam_on_image(img, grayscale_cam, use_rgb=True)

plt.figure(figsize=(6,6))
plt.imshow(visualization)
plt.title(f"True Label: {full_dataset.classes[label]}")
plt.axis("off")
plt.show()
